In [1]:
import numpy as np
from stochastic_volatility_models.src.core.underlying import Underlying
from stochastic_volatility_models.src.core.options import VolatilitySurface
from stochastic_volatility_models.src.core.pricing_models import PricingModel

2024-07-26 12:49:07.401 | INFO     | stochastic_volatility_models.config:initialise:30 - Loaded module `stochastic_volatility_models` from project directory path `/Users/mayurankv/Documents/Mayuran/Programming/Projects/Academic/Imperial College London/MSc Statistics/Dissertation/Project/modules`


In [2]:
spx = Underlying("SPX")
spx_vs = VolatilitySurface(
	underlying=spx,
	expiries=np.array([np.datetime64("2023-03-22")]),
	strikes=np.array([3800, 4000, 4600]),
	monthly=False,
)
time = np.datetime64("2023-03-08")

In [3]:
display(
	spx_vs.empirical_price(
		time=time,
	)
)

,Bid,Ask,Mid
SPXW 230322C3800000,204.3,208.3,206.30
SPXW 230322P3800000,11.5,11.7,11.60
SPXW 230322C4000000,58.9,59.3,59.10
SPXW 230322P4000000,63.5,64.2,63.85
SPXW 230322C4600000,0.0,0.1,0.05
SPXW 230322P4600000,601.5,606.3,603.90


In [4]:
display(
	spx_vs.empirical_pricing_implied_volatility(
		pricing_model=PricingModel(model="Black-Scholes"),
		time=time,
	)
)

2016-10-05


/Users/mayurankv/Documents/Mayuran/Programming/Projects/Academic/Imperial College London/MSc Statistics/Dissertation/.venv/lib/python3.12/site-packages/py_vollib_vectorized/implied_volatility.py:75: UserWarning: Found Below Intrinsic contracts at index [0, 2, 4]
  below_intrinsic, above_max_price = _check_below_and_above_intrinsic(K, F, flag, undiscounted_option_price, on_error)


,Bid,Ask,Mid
SPXW 230322C3800000,NaN,NaN,NaN
SPXW 230322P3800000,0.671808,0.674283,0.673048
SPXW 230322C4000000,NaN,NaN,NaN
SPXW 230322P4000000,0.884165,0.887827,0.885998
SPXW 230322C4600000,NaN,NaN,NaN
SPXW 230322P4600000,2.065095,2.080952,2.073023
